In [1]:
import numpy as np
import matplotlib.pyplot as plt
#%config InlineBackend.figure_format = 'retina'  # For sharper figures, but it takes more time
import scipy as sp
from copy import deepcopy 

from lisatools.utils.constants import *
from lisatools.sensitivity  import SensitivityMatrix, AET1SensitivityMatrix, AE1SensitivityMatrix
from lisatools.analysiscontainer import AnalysisContainer
from lisatools.datacontainer import DataResidualArray

from bbhx.waveforms.phenomhm import PhenomHMAmpPhase
from bbhx.waveformbuild import BBHWaveformFD
from bbhx.utils.interpolate import CubicSplineInterpolant

import noise_generation as noise_generation
from tools.LISASimulator import LISASimulator, signal_time_to_freq_domain
from tools.likelihood import get_dh

No CuPy or GPU PhenomHM module.
No CuPy or GPU interpolation available.
No CuPy
No CuPy or GPU response available.


Generating Simulated LISA data with colored noise and MBHB signal(s)

In [2]:
Tobs = YRSID_SI
dt = 5.

wave_gen = BBHWaveformFD(amp_phase_kwargs=dict(run_phenomd=False))
sim = LISASimulator(Tobs=Tobs, dt=dt, wave_gen=wave_gen, )

f_ref   = np.array([0.0, 0.0])
phi_ref = np.array([0.0, 1.0])
m1      = np.array([1e6, 8e5])
m2      = np.array([5e5, 2e5])
a1      = np.array([0.2, 0.8])
a2      = np.array([0.4, 0.7])
dist    = np.array([10e3, 10e3])  * PC_SI * 1e6 # 3e3 in Mpc
inc     = np.array([np.pi/3, np.pi/4])
beta    = np.array([np.pi/4., np.pi/5])
lam     = np.array([np.pi/5., np.pi/6])
psi     = np.array([np.pi/6., np.pi/7])
t_ref   = np.array([0.5, 0.8]) * YRSID_SI  # in the SSB reference frame

parameters = np.array([m1, m2, a1, a2, dist, phi_ref, f_ref, inc, lam, beta, psi, t_ref])

modes = [(2,2), (2,1), (3,3), (3,2), (4,4), (4,3)]
waveform_kwargs = dict(direct=False, fill=True, squeeze=False, length=1024)

signal_with_noise_t = sim(seed = 42, parameters=parameters.T[0], modes=modes, waveform_kwargs=waveform_kwargs)

/opt/anaconda3/envs/lisa_env/lib/python3.12/site-packages/lisatools/detector.py:633: RuntimeWarning: divide by zero encountered in divide
  Sa_a = Sa_a_in * (1.0 + (0.4e-3 / frq) ** 2) * (1.0 + (frq / 8e-3) ** 4)
/opt/anaconda3/envs/lisa_env/lib/python3.12/site-packages/lisatools/detector.py:635: RuntimeWarning: divide by zero encountered in power
  Sa_d = Sa_a * (2.0 * np.pi * frq) ** (-4.0)
/opt/anaconda3/envs/lisa_env/lib/python3.12/site-packages/lisatools/detector.py:637: RuntimeWarning: invalid value encountered in multiply
  Sa_nu = Sa_d * (2.0 * np.pi * frq / C_SI) ** 2
/opt/anaconda3/envs/lisa_env/lib/python3.12/site-packages/lisatools/detector.py:642: RuntimeWarning: divide by zero encountered in divide
  Soms_d = Soms_d_in * (1.0 + (2.0e-3 / f) ** 4)
/opt/anaconda3/envs/lisa_env/lib/python3.12/site-packages/lisatools/detector.py:644: RuntimeWarning: invalid value encountered in multiply
  Soms_nu = Soms_d * (2.0 * np.pi * frq / C_SI) ** 2


In [3]:
snr_from_lisatools = sim.SNR_optimal_lisatools()
snr_my_code = sim.SNR_optimal()
print(snr_from_lisatools, snr_my_code)

[3530.17281808] [3530.17281808]


In [4]:
sim.signal_with_noise_t.shape[2]

6311628

In [5]:
sim.signal_with_noise_t

np.fft.rfft(sim.signal_with_noise_t, axis=-1)[0]

array([[ 2.79037283e+07+0.00000000e+00j,  3.75542563e-10-5.14973584e-10j,
         4.83656882e-10-5.62939548e-10j, ...,
         1.13318355e-11+5.58519896e-12j, -4.75170395e-14+1.01652022e-12j,
        -1.31900268e-10+0.00000000e+00j],
       [ 7.34455874e+07+0.00000000e+00j,  1.04099369e-09-1.42255681e-09j,
         1.31827044e-09-1.53324993e-09j, ...,
         2.74562861e-11+1.33526566e-11j,  1.05072177e-12+1.34621198e-11j,
        -3.35710126e-10+0.00000000e+00j],
       [ 1.56913451e+07+0.00000000e+00j,  2.20974534e-10-2.98847057e-10j,
         2.72874189e-10-3.23823290e-10j, ...,
         6.89360284e-12+3.03232194e-12j,  2.33213521e-12+1.45283669e-12j,
        -7.40130179e-11+0.00000000e+00j]], shape=(3, 3155815))

In [6]:
data = sim.signal_with_noise_f
f_array = np.fft.rfftfreq(sim.signal_with_noise_t.shape[2])  # returns the correct frequency array for the signal with noise
f_array[0] = f_array[1]  # avoid zero frequency
data[0]

array([[ 2.79037283e+07+0.00000000e+00j,  3.75542563e-10-5.14973584e-10j,
         4.83656882e-10-5.62939548e-10j, ...,
         1.13318355e-11+5.58519896e-12j, -4.75170395e-14+1.01652022e-12j,
        -1.31900268e-10+0.00000000e+00j],
       [ 7.34455874e+07+0.00000000e+00j,  1.04099369e-09-1.42255681e-09j,
         1.31827044e-09-1.53324993e-09j, ...,
         2.74562861e-11+1.33526566e-11j,  1.05072177e-12+1.34621198e-11j,
        -3.35710126e-10+0.00000000e+00j],
       [ 1.56913451e+07+0.00000000e+00j,  2.20974534e-10-2.98847057e-10j,
         2.72874189e-10-3.23823290e-10j, ...,
         6.89360284e-12+3.03232194e-12j,  2.33213521e-12+1.45283669e-12j,
        -7.40130179e-11+0.00000000e+00j]], shape=(3, 3155815))

In [7]:
sim.df, len(sim.signal_with_noise_t[0,0])*sim.dt, sim.Tobs, sim.Tobs / sim.dt, sim.time.shape, sim.freq

(np.float64(3.168754559045622e-08),
 31558140.0,
 31558140.0,
 6311628.0,
 (6311628,),
 array([0.00000000e+00, 3.16875456e-08, 6.33750912e-08, ...,
        9.99999366e-02, 9.99999683e-02, 1.00000000e-01], shape=(3155815,)))

In [8]:
f_array[1] - f_array[0] # should be equal to sim.df

np.float64(0.0)

In [9]:
template = wave_gen(*parameters.T[0], freqs=f_array, modes=modes, **waveform_kwargs)[0]
get_dh(data, template, sens_mat=sim.sens_mat, df=f_array[2] - f_array[1] )

np.float64(-503215960796.56573)

In [10]:

template = DataResidualArray(template, f_arr=f_array)
data = DataResidualArray(data[0], f_arr=f_array)
analysis = AnalysisContainer(data_res_arr=data, sens_mat=sim.sens_mat)
analysis.template_inner_product(template=template)

-503215960796.4901

Inner product changes from 44374.39799894732 to -503215960796.4901 if you remove the line self.freq[0] = self.freq[1] in LISASimulator